In [1]:
import openai, json

client = openai.OpenAI()
messages = []

In [ ]:
def get_weather(city):
    return '33 degrees Celsius, Sunny!!'

FUNCTION_MAP = {
    'get_weather': get_weather
}

In [3]:
# tool의 schema
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather for",
                    },
                },
                "required": ["city"],
            },
        }
    }
]

In [ ]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        # ai 가 코드 실행 요청을 한 것을 메시지 메모리에 추가
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [{
                "id": tool_call.id,
                "type": "function",
                "function": {
                    "name": tool_call.function.name,
                    "arguments": tool_call.function.arguments
                }
            } for tool_call in message.tool_calls]
        })
    
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            
            print(f"Function Call: {function_name} with arguments {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
            
            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} got result: {result}")
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result
            })
        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

def call_ai():
    # 메시지 memory 와 tool schema 를 AI 에게 전달해 호출
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS
    )
    process_ai_response(response.choices[0].message)


In [5]:
while True:
    # user가 메세지를 보냄
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        # user 메세지를 메모리에 추가
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: my name is daeyeol
[DEBUG] call_ai called
[DEBUG] process_ai_response called
[DEBUG] No tool calls, printing response
AI: Nice to meet you, Daeyeol! How can I assist you today?
User: what is my name?
[DEBUG] call_ai called
[DEBUG] process_ai_response called
[DEBUG] No tool calls, printing response
AI: Your name is Daeyeol.
User: how is the weather in spain?
[DEBUG] call_ai called
[DEBUG] process_ai_response called
[DEBUG] Tool calls detected: 1
Function Call: get_weather with arguments {"city":"Spain"}
Ran get_weather with args {'city': 'Spain'} got result: 33 degrees Celsius, Sunny
[DEBUG] call_ai called
[DEBUG] process_ai_response called
[DEBUG] No tool calls, printing response
AI: The weather in Spain is currently 33 degrees Celsius and sunny.


In [6]:
messages

[{'role': 'user', 'content': 'my name is daeyeol'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Daeyeol! How can I assist you today?'},
 {'role': 'user', 'content': 'what is my name?'},
 {'role': 'assistant', 'content': 'Your name is Daeyeol.'},
 {'role': 'user', 'content': 'how is the weather in spain?'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_vCsi0QE6pkZvyF4hkR7ZiAwg',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Spain"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_vCsi0QE6pkZvyF4hkR7ZiAwg',
  'name': 'get_weather',
  'content': '33 degrees Celsius, Sunny'},
 {'role': 'assistant',
  'content': 'The weather in Spain is currently 33 degrees Celsius and sunny.'}]